# Neural Network Using All Available Data

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('data/processed_data.csv')

In [7]:
# First split: 90% train+val, 10% test
train_val, test = train_test_split(data, test_size=0.1, random_state=42)

# Second split: 70% train, 20% val (from the original dataset)
# 70/90 = 0.778 (train should be 77.8% of the train_val set)
train, val = train_test_split(train_val, test_size=0.222, random_state=42)

# Verify the shapes
print(f"Original: {data.shape}")
print(f"Training: {train.shape} ({len(train)/len(data):.1%})")
print(f"Validation: {val.shape} ({len(val)/len(data):.1%})")
print(f"Testing: {test.shape} ({len(test)/len(data):.1%})")

Original: (28800, 61)
Training: (20165, 61) (70.0%)
Validation: (5755, 61) (20.0%)
Testing: (2880, 61) (10.0%)
